***in this notebook we have created new features like 'session duration' and 'session_gap' with pyspark because the dataset is very voulimineux. Pyspark is the most optimal for us***




In [ ]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [66.7 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.e

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ColabPySpark").getOrCreate()

print("PySpark Version:", spark.version)


PySpark Version: 3.5.4


In [ ]:
df_spark = spark.read.csv("/content/drive/MyDrive/mooc_data/df_train_log_course.csv", header=True, inferSchema=True)
df_spark.show(5)


+---------+--------+--------------------+--------------------+-----------+-------------------+--------+
|enroll_id|username|           course_id|          session_id|     action|               time|category|
+---------+--------+--------------------+--------------------+-----------+-------------------+--------+
|      772|    5981|course-v1:Tsinghu...|d8a9b787fa69063c3...|click_about|2015-09-27 15:42:59|     art|
|      772|    5981|course-v1:Tsinghu...|d8a9b787fa69063c3...| click_info|2015-09-27 15:43:12|     art|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...|pause_video|2015-10-19 19:37:42|     art|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...| load_video|2015-10-19 19:33:27|     art|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...| play_video|2015-10-19 19:33:30|     art|
+---------+--------+--------------------+--------------------+-----------+-------------------+--------+
only showing top 5 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, unix_timestamp, col




df_spark = df_spark.withColumn("time", col("time").cast("timestamp"))


session_durations = df_spark.groupBy("username", "enroll_id", "session_id").agg(
    min("time").alias("session_start"),
    max("time").alias("session_end")
)


session_durations = session_durations.withColumn("session_duration",
    unix_timestamp(col("session_end")) - unix_timestamp(col("session_start"))
)


df_spark = df_spark.join(session_durations.select("username", "enroll_id", "session_id", "session_duration"),
                         on=["username", "enroll_id", "session_id"], how="left")


df_spark.select("username", "enroll_id", "session_id", "session_duration").distinct().show(10)




+--------+---------+--------------------+----------------+
|username|enroll_id|          session_id|session_duration|
+--------+---------+--------------------+----------------+
|       5|   136060|c4d05fdd4af11ea45...|           10468|
|       5|   189082|7a4aa45ee44784b02...|             166|
|       5|   246788|c4d05fdd4af11ea45...|          605093|
|       5|   258790|d1d739518c0b8353a...|               0|
|       7|   463935|30c5f66078ba4d852...|           92406|
|      26|    39180|7e4485af421089caa...|              50|
|      26|   257780|be4255148c88e179b...|            1115|
|      26|   275289|9244e25da6e21ef80...|              49|
|      32|   400456|5e8b851a19fa74976...|            2636|
|      35|    12347|00d0223d8e65ff0c5...|               0|
+--------+---------+--------------------+----------------+
only showing top 10 rows



In [ ]:
df_spark.show(5)

+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+
|username|enroll_id|          session_id|           course_id|          action|               time|category|session_duration|
+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+
|    5981|      772|d8a9b787fa69063c3...|course-v1:Tsinghu...|     click_about|2015-09-27 15:42:59|     art|              13|
|    5981|      772|d8a9b787fa69063c3...|course-v1:Tsinghu...|      click_info|2015-09-27 15:43:12|     art|              13|
|  102691|   260571|365fb04030044ef58...|course-v1:Tsinghu...|     pause_video|2016-04-29 18:06:59| Unknown|           77273|
|  102691|   260571|365fb04030044ef58...|course-v1:Tsinghu...|click_courseware|2016-04-30 12:43:15| Unknown|           77273|
|  102691|   260571|365fb04030044ef58...|course-v1:Tsinghu...|click_courseware|2016-04-30 12:43:22| Unknown|          

In [ ]:
from pyspark.sql.functions import min, max, unix_timestamp, lag, col
from pyspark.sql.window import Window


session_gap_df = df_spark.groupBy("username", "enroll_id", "session_id").agg(
    min("time").alias("session_start"),
    max("time").alias("session_end")
)


window_spec = Window.partitionBy("username", "enroll_id").orderBy("session_start")


session_gap_df = session_gap_df.withColumn("prev_session_end", lag("session_end").over(window_spec))

session_gap_df = session_gap_df.withColumn("session_gap",
    unix_timestamp(col("session_start")) - unix_timestamp(col("prev_session_end"))
)


session_gap_df = session_gap_df.fillna({"session_gap": 0})


df_spark = df_spark.join(session_gap_df.select("username", "enroll_id", "session_id", "session_gap"),
                         on=["username", "enroll_id", "session_id"], how="left")


df_spark.distinct().show(10)




+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+-----------+
|username|enroll_id|          session_id|           course_id|          action|               time|category|session_duration|session_gap|
+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+-----------+
|       5|   136060|c4d05fdd4af11ea45...|TsinghuaX/6051010...|close_courseware|2015-10-12 17:34:13|business|           10468|          0|
|       5|   136060|c4d05fdd4af11ea45...|TsinghuaX/6051010...|      seek_video|2015-10-12 14:40:02|business|           10468|          0|
|       5|   136060|c4d05fdd4af11ea45...|TsinghuaX/6051010...|      seek_video|2015-10-12 14:40:03|business|           10468|          0|
|       5|   136060|c4d05fdd4af11ea45...|TsinghuaX/6051010...|      seek_video|2015-10-12 14:40:05|business|           10468|          0|
|       5|   136060|c4d05fdd4af11e

In [ ]:
df_spark.show(5)

+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+-----------+
|username|enroll_id|          session_id|           course_id|          action|               time|category|session_duration|session_gap|
+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+-----------+
|    5981|      772|d8a9b787fa69063c3...|course-v1:Tsinghu...|     click_about|2015-09-27 15:42:59|     art|              13|          0|
|    5981|      772|d8a9b787fa69063c3...|course-v1:Tsinghu...|      click_info|2015-09-27 15:43:12|     art|              13|          0|
|  102691|   260571|365fb04030044ef58...|course-v1:Tsinghu...|     pause_video|2016-04-29 18:06:59| Unknown|           77273|          0|
|  102691|   260571|365fb04030044ef58...|course-v1:Tsinghu...|click_courseware|2016-04-30 12:43:15| Unknown|           77273|          0|
|  102691|   260571|365fb04030044e

In [ ]:
from pyspark.sql.functions import count, col, when


action_count_df = df_spark.groupBy("username", "enroll_id", "session_id").agg(
    count("action").alias("action_count")
)


df_spark = df_spark.join(action_count_df, on=["username", "enroll_id", "session_id"], how="left")


df_spark = df_spark.withColumn("action_frequency",
    when(col("session_duration") > 0, col("action_count") / col("session_duration"))
    .otherwise(0)  # Si `session_duration = 0`, on met `action_frequency = 0`
)


df_spark.show(10)




+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+-----------+------------+-------------------+
|username|enroll_id|          session_id|           course_id|          action|               time|category|session_duration|session_gap|action_count|   action_frequency|
+--------+---------+--------------------+--------------------+----------------+-------------------+--------+----------------+-----------+------------+-------------------+
|    5981|      772|d8a9b787fa69063c3...|course-v1:Tsinghu...|     click_about|2015-09-27 15:42:59|     art|              13|          0|           2|0.15384615384615385|
|    5981|      772|d8a9b787fa69063c3...|course-v1:Tsinghu...|      click_info|2015-09-27 15:43:12|     art|              13|          0|           2|0.15384615384615385|
|  102691|   260571|365fb04030044ef58...|course-v1:Tsinghu...|     pause_video|2016-04-29 18:06:59| Unknown|           77273|          0|        

In [ ]:
df_spark.write.csv("dfspark_train_log_featured_final.csv", header=True, mode="overwrite")


In [ ]:
import os
print(os.listdir("/content/"))  # Vérification

['.config', 'drive', 'dfspark_train_log_featured_final.csv', 'sample_data']


In [ ]:
!cat /content/dfspark_train_log_featured_final.csv/*.csv > /content/dfspark_train_log_final.csv


In [ ]:
from google.colab import files
#files.download("/content/dfspark_train_log_final.csv")
